In [ ]:
%pip install gradio
%pip install --upgrade transformers
%pip install plotly
%pip install kaleido

!mkdir my_models
!mkdir my_models/phi_3_vision

In [ ]:
import torch

from AIModel import *

phi3v = AIModel()

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from prompts import *
from Validator import *
   
validator = Validator()
"""
validator.validation(phi3v)

categories = ['Pie chart', 'Bar chart', 'Line chart', 'Trash']

print(validator.bad)
print(validator.confusion_matrix)
df_cm = pd.DataFrame(validator.confusion_matrix, index = [i for i in categories],
                  columns = [i for i in categories])
#plt.figure(figsize = (10,7))
heat_map = sns.heatmap(df_cm, annot=True, fmt='g')

fig = heat_map.get_figure()
fig.savefig("out.png")
heat_map
"""

In [ ]:
from BarChart import *
from LineChart import *
from PieChart import *

bar_chart = BarChart(phi3v)
line_chart = LineChart(phi3v)
pie_chart = PieChart(phi3v)

In [ ]:
class GUC:
    def __init__(self):
        pass
    def respond(image):
        chart_type = phi3v.get_response(orchestrator_prompt, image).lower()
        img_trash = Image.open("/notebooks/image_2024-07-26_174626009.png")
        
        if "pie chart" in chart_type:
            return pie_chart.give_chart(image)
        elif "bar chart" in chart_type:
            return bar_chart.give_chart(image)
        elif "line chart" in chart_type:
            return line_chart.give_chart(image)
        elif "trash" in chart_type:
            return img_trash
        else:
            raise ValueError("Unknown chart")

In [ ]:
import gradio as gr

example1 = Image.open("/notebooks/dataset/pie/image_3800.jpg")
example2 = Image.open("/notebooks/dataset/bar/image_121.jpg")
example3 = Image.open("/notebooks/dataset/line/image_1019.jpg")

demo = gr.Interface (
    fn=GUC.respond,
    inputs=[gr.Image(label="Upload your graph here")],
    outputs=[gr.Image(label="Processed graph")],
    title="Graph Understanding Copilot",
    description="Upload an image to see the generated chart.",
    theme=gr.Theme.from_hub("gradio/soft"),
    examples=[[example1], [example2], [example3]],
    example_labels=["Pie", "Bar", "Line"],
    examples_per_page=3,
    allow_flagging="never"
)

demo.launch(share=True)